In [2]:
import subprocess
import zipfile
import os

subprocess.run(["kaggle", "competitions", "download", "-c", "llm-detect-ai-generated-text"])

with zipfile.ZipFile("llm-detect-ai-generated-text.zip") as zf:
    zf.extractall("dataset")

os.remove("./llm-detect-ai-generated-text.zip")

In [3]:
import pandas as pd

train_essays = pd.DataFrame(pd.read_csv("./dataset/train_essays.csv"))
train_prompts = pd.DataFrame(pd.read_csv("./dataset/train_prompts.csv"))
test_essays = pd.DataFrame(pd.read_csv("./dataset/test_essays.csv"))

In [4]:
test_essays

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [5]:
import re

# "text" 列の各行に対して "\n\n" を削除する関数を定義
def remove_newline(text):
    text = re.sub(r"\n\n", "", text)
    return text.replace("\n\n", " ").replace(r"\\n\\n", "")

# "text" 列に対して上記の関数を適用
train_essays["text"] = train_essays["text"].apply(remove_newline)

# 結果を表示
print(train_essays["text"][0])

Cars. Cars have been around since they became famous in the 1900s, when Henry Ford created and built the first ModelT. Cars have played a major role in our every day lives since then. But now, people are starting to question if limiting car usage would be a good thing. To me, limiting the use of cars might be a good thing to do.In like matter of this, article, "In German Suburb, Life Goes On Without Cars," by Elizabeth Rosenthal states, how automobiles are the linchpin of suburbs, where middle class families from either Shanghai or Chicago tend to make their homes. Experts say how this is a huge impediment to current efforts to reduce greenhouse gas emissions from tailpipe. Passenger cars are responsible for 12 percent of greenhouse gas emissions in Europe...and up to 50 percent in some carintensive areas in the United States. Cars are the main reason for the greenhouse gas emissions because of a lot of people driving them around all the time getting where they need to go. Article, "Pa

In [6]:
from datasets import Dataset


train_essays_dataset =  Dataset.from_pandas(train_essays[["text", "generated"]])
dataset_split_test = train_essays_dataset.train_test_split(test_size=0.2, shuffle=True)

print(dataset_split_test["train"][0])

{'text': 'There are many advantages to limiting car usage in our community. Other countries such as France, Germany, and Colombia are home to cities that are working towards cutting down the use of personal automobiles. Many of the people in these places find that using alternative transportation means proves to be less stressful. Studies even show that fewer Americans are purchasing cars for themselves. Now is the perfect time to join in with Vauban, Bogota, and New York and spend less time in our cars.It may seem hard to believe, but in Germany, there\'s a suburban area where residents live without their own cars. According to "In German Suburb, Life Goes On Without Cars" by Elisabeth Rosenthal, the streets of Vauban, Germany remain "carfree" aside from some public transport. The article states that "70 percent of vaughn\'s families do not own cars, and 57 percent sold a car to move here.". Heidrun Walter was quoted in the excerpt saying "When I had a car I was always tense. I\'m muc

c:\Users\yuhei\anaconda3\envs\pytorch_env\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
print(tokenizer.special_tokens_map)
print("bos_token :", tokenizer.bos_token, ",", tokenizer.bos_token_id)
print("eos_token :", tokenizer.eos_token, ",", tokenizer.eos_token_id)
print("unk_token :", tokenizer.unk_token, ",", tokenizer.unk_token_id)
print("pad_token :", tokenizer.pad_token, ",", tokenizer.pad_token_id)


{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}
bos_token : <s> , 0
eos_token : </s> , 2
unk_token : <unk> , 3
pad_token : <pad> , 1


In [8]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

from peft import (
    PeftModel,
    LoraConfig,
    get_peft_model,
)


# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
masked_base_model = AutoModelForMaskedLM.from_pretrained("roberta-base")

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# PEFTのコンフィグを設定
# PEFTの設定
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "key",  "value", "out_proj", "dense"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",  # テキスト分類の場合は"SEQ_CLS"となる
)

# PEFTモデルを作成
masked_model = get_peft_model(masked_base_model, peft_config)
print_trainable_parameters(masked_model)



trainable params: 1339392 || all params: 126036825 || trainable%: 1.0626989373938927


In [9]:
from torch.utils.data import Dataset, DataLoader


class MyDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        super().__init__()
        self.dataset = dataset
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        data = self.dataset.iloc[idx]
        labels = data["generated"]
        tokenized_texts = self.tokenizer(data["text"], return_tensors='pt',truncation=True, max_length=512, padding='max_length')

        return tokenized_texts, labels

In [10]:
class EarlyStopping:
    def __init__(self, patience):
        self.check_count = 0
        self.patience = patience

    def checkCount(self, _bool):
        if _bool:
            self.check_count = 0
        else:
            self.check_count += 1

        if self.check_count == self.patience:
            return 0
        else:
            return None
early_stopping = EarlyStopping(patience=3)

In [12]:
import os
from tqdm import tqdm
import random
import torch
import pandas as pd
import sklearn
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from torch.nn import functional as F
import torch.nn as nn
import torch.optim as optim
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorWithPadding,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    pipeline,
    logging,
    RobertaTokenizer,
    RobertaForSequenceClassification,
    RobertaConfig
)
"C:/Users/yuhei/VScode/python/LLM_Detect_AI_Generated_Text/dataset/"
train_dataframe = pd.DataFrame(pd.read_csv("C:/Users/yuhei/VScode/python/LLM_Detect_AI_Generated_Text/dataset/train_essays.csv"))
train_dataframe1 = pd.DataFrame(pd.read_csv("C:/Users/yuhei/VScode/python/LLM_Detect_AI_Generated_Text/dataset/Training_Essay_Data.csv"))
train_dataframe2 = pd.DataFrame(pd.read_csv("C:/Users/yuhei/VScode/python/LLM_Detect_AI_Generated_Text/dataset/Wikipedia.csv"))
train_dataframe3 = pd.DataFrame(pd.read_csv("C:/Users/yuhei/VScode/python/LLM_Detect_AI_Generated_Text/dataset/data_set.csv"))
train_dataframe4 = pd.DataFrame(pd.read_csv("C:/Users/yuhei/VScode/python/LLM_Detect_AI_Generated_Text/dataset/train_essays_RDizzl3_seven_v2.csv"))
train_dataframe5 = pd.DataFrame(pd.read_csv("C:/Users/yuhei/VScode/python/LLM_Detect_AI_Generated_Text/dataset/dataset.csv"))

tmp_train_dataframe = train_dataframe.drop(columns=["id", "prompt_id"]).rename(columns={"generated": "labels"})
tmp_train_dataframe.sample(frac=1).reset_index(drop=True)

dataset_length = len(tmp_train_dataframe)

tmp_test_dataframe = tmp_train_dataframe.iloc[int(dataset_length * 0.8):]
tmp_train_dataframe = tmp_train_dataframe.iloc[:int(dataset_length * 0.8)]


tmp_train_dataframe1 = train_dataframe1.rename(columns={"generated": "labels"})
tmp_train_dataframe2 = train_dataframe2.drop(columns=["Human"]).rename(columns={"Text":"text", "AI": "labels"})
tmp_train_dataframe3 = train_dataframe3.drop(columns=["title", "ai_generated"]).rename(columns={"abstract":"text", "is_ai_generated": "labels"})
tmp_train_dataframe4 = train_dataframe4.rename(columns={"label": "labels"})
tmp_train_dataframe5 = train_dataframe5.drop(columns=["Unnamed: 0"]).rename(columns={"class": "labels"})
tmp_train_dataframe5["labels"] = tmp_train_dataframe5["labels"].apply(lambda x: 1 if x == "AI-Generated-Text" else 0)

def concat_dataframe(*dataframes):
    result_dataframe = pd.concat(dataframes, axis=0)
    return result_dataframe

train_essays = concat_dataframe(
    tmp_train_dataframe,
    tmp_train_dataframe1,
    tmp_train_dataframe4,
)

# 特定の文字を取り除く
target_word = "\n"
train_essays['text'] =train_essays['text'].str.replace(target_word, '')

from tqdm import tqdm

def max_token_count(dataframe, model_name="roberta-base"):
    # トークナイザーの読み込み
    token_counts_list = []
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    tokenizer.model_max_length = 1027

    # トークン数を計算してリストに保存

    for text in tqdm(dataframe["text"]):
      token_counts_list.append(len(tokenizer.encode(text)))

    return max(token_counts_list), token_counts_list

max_seq_length, token_counts_list = max_token_count(train_essays)

lower_max_length_list = [count <= 1024 for count in token_counts_list]

filterd_dataframe = train_essays[lower_max_length_list]

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=1024):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe['text'].iloc[idx]
        label = self.dataframe['labels'].iloc[idx]

        # Tokenize and encode the text
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        # Extract input_ids, attention_mask, and convert label to tensor
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        label_tensor = torch.tensor(label)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label_tensor
        }
    
# データを訓練用と評価用に分割
train_dataframe, eval_dataframe = train_test_split(filterd_dataframe, test_size=0.1, random_state=42)

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
tokenizer.model_max_length = 1027
batch_size = 2
gradient_accumulation_size = 64
check_steps = gradient_accumulation_size / batch_size

train_dataset = CustomDataset(train_dataframe, tokenizer)
eval_dataset = CustomDataset(eval_dataframe, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=True)

config = RobertaConfig(
    attention_probs_dropout_prob=0.1,
    hidden_act="gelu",
    hidden_dropout_prob=0.1,
    hidden_size=768,
    initializer_range=0.2,
    intermediate_size=3072,
    max_position_embeddings=1027,
    num_attention_heads=12,
    position_embedding_type="absolute",
    type_vocab_size=1,
    pad_token_id=1,
    num_hidden_layers=12,
    output_attentions=False,
    vocab_size=50265

)

model = RobertaForSequenceClassification(config)

class EarlyStopping:
    def __init__(self, patience):
        self.check_count = 0
        self.patience = patience

    def checkCount(self, _bool):
        if _bool:
            self.check_count = 0
        else:
            self.check_count += 1

        if self.check_count == self.patience:
            return 0
        else:
            return None
        
patience = 5
early_stopping = EarlyStopping(patience=patience)

device = torch.device("cuda")

total_samples = len(train_dataset)
class_0_samples = len(train_dataframe[train_dataframe["labels"] == 0])
class_1_samples = len(train_dataframe[train_dataframe["labels"] == 1])

# クラスごとの重みを計算する。
weight_class_0 = total_samples / class_0_samples
weight_class_1 = total_samples / class_1_samples

# 重みをテンソルに変換
weights = torch.tensor([weight_class_0, weight_class_1]).to(device)


criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
model = model.to(device)

def compute_metrics(label_ids, predictions):
    # AUC-ROCの計算
    probabilities = F.softmax(predictions, dim=-1).cpu().detach().numpy()
    auc_roc = roc_auc_score(label_ids.cpu().detach().numpy(), probabilities[:, 1])  # 2クラス分類の場合、クラス1の確率を使います

    return auc_roc

def train(model, train_dataloader, optimizer, criterion, check_steps, device="cuda"):
    epoch_loss = 0.0
    sum_accumulation_loss = 0.0

    #epoch_auc_roc = 0.0

    model.train()

    for batch, data in tqdm(enumerate(train_dataloader)):
        input_ids =data["input_ids"].to(device)
        attention_mask = data["attention_mask"].to(device)
        labels =  data["labels"].to(device)


        outputs = model(input_ids, attention_mask=attention_mask)

        loss = criterion(outputs.logits, labels)
        accumulation_loss = loss / check_steps
        accumulation_loss.backward()
        sum_accumulation_loss += accumulation_loss.item()

        #auc_roc = compute_metrics(labels, outputs.logits)

        if (batch + 1) % check_steps == 0:

            optimizer.step()
            optimizer.zero_grad()

            print()
            print("Gradient Accumulation")
            print(f"Step: Train Iteration: {batch + 1}/{len(train_dataloader)} loss: {sum_accumulation_loss}")
            sum_accumulation_loss = 0.0

        epoch_loss += loss.item()
        #epoch_auc_roc += auc_roc

        print()
        #print(f"Step: Train Iteration: {batch + 1}/{len(train_dataloader)} loss: {loss.item()}")
        #print(f"Step: Train Iteration: {batch + 1} loss: {loss.item()}, auc_roc: {auc_roc.item()}")

    return epoch_loss / (batch + 1)#, epoch_auc_roc / (batch + 1)

def evaluate(model, eval_dataloader, criterion, device="cuda"):
    epoch_loss = 0.0
    #epoch_auc_roc = 0.0

    model.eval()

    with torch.no_grad():
        for batch, data in tqdm(enumerate(eval_dataloader)):
            input_ids =data["input_ids"].to(device)
            attention_mask = data["attention_mask"].to(device)
            labels =  data["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)

            loss = criterion(outputs.logits, labels)
            #auc_roc = compute_metrics(outputs.logits, labels)

            epoch_loss += loss.item()
            #epoch_auc_roc += auc_roc

            print()
            print(f"Step: Eval Iteration: {batch + 1}/{len(eval_dataloader)} loss: {loss.item()}")
            #print(f"Step: Eval Iteration: {batch + 1} loss: {loss.item()}, auc_roc: {auc_roc.item()}")

    return epoch_loss / (batch + 1)#, epoch_auc_roc / (batch + 1)

train_loss_list = []
#train_auc_roc_list = []

eval_loss_list = []
#eval_auc_roc_list = []

epochs = 20
min_loss = 1000
for epoch in range(epochs):

    epoch_train_loss = train(model, train_dataloader, optimizer, criterion, check_steps)
    print()
    print("-"*100)
    print(f"Step: Train Epoch: {epoch + 1}/{epochs}, loss: {epoch_train_loss}")
    #print(f"Step: Train Epoch: {epoch + 1}/{epochs}, loss: {epoch_train_loss}, auc_roc: {epoch_train_auc_roc}")
    print("-"*100)

    epoch_eval_loss = evaluate(model, eval_dataloader, criterion)
    print()
    print("-"*100)
    print(f"Step: Eval Epoch: {epoch + 1}/{epochs}, loss: {epoch_eval_loss}")
    #print(f"Step: Eval Epoch: {epoch + 1}/{epochs}, loss: {epoch_eval_loss}, auc_roc: {epoch_eval_auc_roc}")
    print("-"*100)

    train_loss_list.append(epoch_train_loss)
    #train_auc_roc_list.append(epoch_train_auc_roc)

    eval_loss_list.append(epoch_eval_loss)
    #eval_auc_roc_list.append(epoch_eval_auc_roc)

    if epoch_eval_loss < min_loss:
        min_loss = epoch_eval_loss
        model.save_pretrained("./roberta_classification_model/")
        early_stopping.checkCount(True)

    else:
      check = early_stopping.checkCount(False)
      if check == 0:
        print("Early Stopping !")
        break


RuntimeError: Expected target size [16, 50265], got [16]

In [67]:

from transformers import RobertaForSequenceClassification
import torch
from peft import (
    PeftModel,
    LoraConfig,
    get_peft_model,
)

# ベースのRobertaモデルを読み込む
base_model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# PEFTのコンフィグを設定
# PEFTの設定
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "key",  "value", "out_proj", "dense"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",  # テキスト分類の場合は"SEQ_CLS"となる
)

# PEFTモデルを作成
model = get_peft_model(base_model, peft_config)
print_trainable_parameters(model)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should pr

trainable params: 2548260 || all params: 126603300 || trainable%: 2.0127911357760815


In [68]:
from torch.utils.data import Dataset, DataLoader


class MyDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        super().__init__()
        self.dataset = dataset
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        data = self.dataset.iloc[idx]
        labels = data["generated"]
        tokenized_texts = self.tokenizer(data["text"], return_tensors='pt',truncation=True, max_length=512, padding='max_length')

        return tokenized_texts, labels
    
train_dataset = MyDataset(train_essays, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

print(len(train_dataset))

1378


In [70]:
class EarlyStopping:
    def __init__(self, patience):
        self.check_count = 0
        self.patience = patience

    def checkCount(self, _bool):
        if _bool:
            self.check_count = 0
        else:
            self.check_count += 1

        if self.check_count == self.patience:
            return 0
        else:
            return None
early_stopping = EarlyStopping(patience=3)

In [71]:
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import roc_auc_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = torch.tensor().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
softmax = nn.Softmax(dim=-1)

batch_size = 16
epochs = 3
output_dir = "./output"
check = None
min_loss = np.inf

folds = StratifiedKFold(n_splits=5)
splits = folds.split(np.zeros(len(train_essays)), train_essays["generated"])

train_loss_lists = []
train_correct_lists = []

val_loss_lists = []
val_correct_lists = []



for fold, (train_idxs, val_idxs) in enumerate(splits):
    model = get_peft_model(base_model, peft_config).to(device)
    train_dataset = MyDataset(train_essays.iloc[train_idxs], tokenizer)
    val_dataset = MyDataset(train_essays.iloc[val_idxs], tokenizer)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    train_loss_list = []
    train_correct_list = []

    val_loss_list = []
    val_correct_list = []

    for epoch in range(epochs):
        for step in ["train", "val"]:
            running_loss = 0.0
            running_correct = 0.0

            if step == "train":
                model.train()
                dataloader = train_dataloader

            else:
                model.eval()
                dataloader = val_dataloader

            for batch, (inputs, labels) in enumerate(dataloader):
                changeable_batch_size = len(labels)
                inputs["input_ids"] = inputs["input_ids"].squeeze(1) # (batch_size, 1, seq_length) -> (batch_size, seq_length)
                inputs = inputs.to(device)
                labels = labels.to(device)

                with torch.set_grad_enabled(step == "train"):
                    outputs = model(**inputs).logits
                    pred = torch.argmax(softmax(outputs), dim=-1)
                    loss = criterion(outputs, labels)
                    correct = torch.sum(pred == labels) / changeable_batch_size

                    if step == "train":
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item()
                running_correct += correct.item()

                if step == "train":
                    print(f"Fold: {fold + 1} Step: Train Epoch: {epoch + 1}/{epochs} Iteration: {batch + 1}/{len(dataloader)} train_loss: {running_loss / (batch + 1)} train_correct {running_correct / (batch + 1)}")

                else:
                    print(f"Fold: {fold + 1} Step: Val Epoch: {epoch + 1}/{epochs} Iteration: {batch + 1}/{len(dataloader)} val_loss: {running_loss / (batch + 1)} val_correct {running_correct / (batch + 1)}")
            
            if step == "train":
                train_loss_list.append(running_loss / (batch + 1))
                train_correct_list.append(running_correct / (batch + 1))
            else:
                val_loss_list.append(running_loss / (batch + 1))
                val_correct_list.append(running_correct / (batch + 1))                

                if running_loss < min_loss:
                    early_stopping.checkCount(True)
                    print("Model Save !")
                    min_loss = running_loss
                    if not os.path.exists(output_dir):
                        os.mkdir(output_dir)
                    torch.save(model.state_dict(), os.path.join(output_dir, "model.pth"))
                else:
                    check = early_stopping.checkCount(False)
                if check == 0:
                    print("Early Stopping !") 
                    break
    train_loss_lists.append(train_loss_list)
    train_correct_lists.append(train_correct_list)
    val_loss_lists.append(val_loss_list)
    val_correct_lists.append(val_correct_list)

c:\Users\yuhei\anaconda3\envs\pytorch_env\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Fold: 1 Step: Train Epoch: 1/5 Iteration: 1/69 train_loss: 0.7375463843345642 train_correct 0.25
Fold: 1 Step: Train Epoch: 1/5 Iteration: 2/69 train_loss: 0.7475179433822632 train_correct 0.125
Fold: 1 Step: Train Epoch: 1/5 Iteration: 3/69 train_loss: 0.7467154661814371 train_correct 0.125
Fold: 1 Step: Train Epoch: 1/5 Iteration: 4/69 train_loss: 0.7467857748270035 train_correct 0.125
Fold: 1 Step: Train Epoch: 1/5 Iteration: 5/69 train_loss: 0.7478360772132874 train_correct 0.1
Fold: 1 Step: Train Epoch: 1/5 Iteration: 6/69 train_loss: 0.7466359833876292 train_correct 0.11458333333333333
Fold: 1 Step: Train Epoch: 1/5 Iteration: 7/69 train_loss: 0.7490234545298985 train_correct 0.11607142857142858
Fold: 1 Step: Train Epoch: 1/5 Iteration: 8/69 train_loss: 0.7491680681705475 train_correct 0.109375
Fold: 1 Step: Train Epoch: 1/5 Iteration: 9/69 train_loss: 0.748780263794793 train_correct 0.1111111111111111
Fold: 1 Step: Train Epoch: 1/5 Iteration: 10/69 train_loss: 0.7455938160419464

KeyboardInterrupt: 